In [1]:

import numpy as np 

a_1 = 1
a_2 = 4
lam = 1.0




a_1 = 1
a_2 = 4

def u(x, a_1, a_2):
    return np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

def u_xx(x, a_1, a_2):
    return - (a_1 * np.pi) ** 2 * np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

def u_yy(x, a_1, a_2):
    return - (a_2 * np.pi) ** 2 * np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

# Forcing
def f(x, a_1, a_2, lam):
    return u_xx(x, a_1, a_2) + u_yy(x, a_1, a_2) + lam * u(x, a_1, a_2)

def operator(u, x1, x2, lam, sigma_x1=1.0, sigma_x2=1.0):
    u_x1 = tf.gradients(u, x1)[0] / sigma_x1
    u_x2 = tf.gradients(u, x2)[0] / sigma_x2
    u_xx1 = tf.gradients(u_x1, x1)[0] / sigma_x1
    u_xx2 = tf.gradients(u_x2, x2)[0] / sigma_x2
    residual = u_xx1 + u_xx2 + lam * u
    return residual



def r(x, a, c):
    return u_tt(x, a, c) - c**2 * u_xx(x, a, c)

def fetch_minibatch( sampler, N):
    X, Y = sampler.sample(N)
    X = (X - mu_X) / sigma_X
    return X, Y


class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name = None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name
    def sample(self, N):
        x = self.coords[0:1,:] + (self.coords[1:2,:]-self.coords[0:1,:])*np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y
    
kernel_size = 300

bc1_coords = np.array([[-1.0, -1.0], [1.0, -1.0]])
bc2_coords = np.array([[1.0, -1.0],[1.0, 1.0]])
bc3_coords = np.array([[1.0, 1.0], [-1.0, 1.0]])
bc4_coords = np.array([[-1.0, 1.0], [-1.0, -1.0]])

dom_coords = np.array([[-1.0, -1.0], [1.0, 1.0]])


# Create boundary conditions samplers
bc1 = Sampler(2, bc1_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC1')
bc2 = Sampler(2, bc2_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC2')
bc3 = Sampler(2, bc3_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC3')
bc4 = Sampler(2, bc4_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC4')
bcs_sampler = [bc1, bc2, bc3, bc4]
ics_sampler = None
res_sampler = Sampler(2, dom_coords, lambda x: f(x, a_1, a_2, lam), name='Forcing')


X, _ = res_sampler.sample(np.int32(1e5))
mu_X, sigma_X = X.mean(0), X.std(0)
mu_x1, sigma_x1 = mu_X[0], sigma_X[0]
mu_x2, sigma_x2 = mu_X[1], sigma_X[1]

X_bc1_batch, _ = fetch_minibatch(bcs_sampler[0],  kernel_size)
X_bc2_batch, _ = fetch_minibatch(bcs_sampler[1],  kernel_size)
X_bc3_batch, _ = fetch_minibatch(bcs_sampler[2],  kernel_size)
X_bc4_batch, _ = fetch_minibatch(bcs_sampler[3],  kernel_size)
X_bc_batch = np.concatenate([X_bc1_batch , X_bc2_batch , X_bc3_batch , X_bc4_batch] , 0)

idx_c = np.random.choice( X_bc_batch.shape[0],  kernel_size, replace=False)
# Fetch residual mini-batch
X_res_batch, _ = fetch_minibatch(res_sampler,  kernel_size)


NameError: name 'f' is not defined